## 一、安装所需的库

In [1]:
!pip install mindnlp

Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple/

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


## 二、加载数据

首先，导入了必要的库，包括 mindnlp.dataset 中的 load_dataset 和用于保证可复现性的 random。接着加载了 IMDB 数据集，并将其划分为训练集和测试集。最后，代码将原始训练集进一步划分为一个新的、较小的训练集（占70%）和一个验证集（占30%）。
这里提到的 IMDB 数据集是一个广泛应用于情感分析或文本分类任务的流行数据集。它包含了从互联网电影数据库 (IMDB) 提取的 50,000 条电影评论，平均分配为 25,000 条用于训练，25,000 条用于测试。每条评论都被标记为正面或负面，这使其成为一个理想的二元数据集，适用于训练能够理解和分类文本中所表达观点的情感分类模型。它在自然语言处理社区被广泛用于比较情感分类模型的性能。

In [2]:
from mindnlp.dataset import load_dataset
import random
random.seed(42)
# 加载完整数据集
imdb_ds = load_dataset('imdb', split=['train', 'test'])
imdb_train = imdb_ds['train']
imdb_test = imdb_ds['test']

imdb_train, imdb_val = imdb_train.split([0.7, 0.3])
print(imdb_train[0])

/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


("'Stanley and Iris' show the triumph of the human spirit. For Stanley, it's the struggle to become literate and realize his potential. For Iris, it's to find the courage to love again after becoming a widow. The beauty of the movie is the dance that Robert DeNiro and Jane Fonda do together, starting and stopping, before each has the skills and courage to completely trust each other and move on. In that sense it very nicely gives us a good view of how life often is, thus being credible. Unlike some other reviewers I found the characters each rendered to be consistent for the whole picture. The supporting cast is also carefully chosen and they add a depth of character that the main characters get added meaning from the supporting performances. All in all an excellent movie. The best thing I take from it is Hope.", 1)


## 三、数据预处理

首先，它定义了一个内部的 tokenize 函数，该函数利用传入的 tokenizer 将文本数据转换为模型所需的 input_ids（词元ID）、token_type_ids（片段ID）和 attention_mask（注意力掩码），同时根据 max_seq_len 控制序列长度。然后，函数会根据 shuffle 参数决定是否打乱数据集顺序，并根据 take_len 参数选择是否只取数据集的一部分。接着，它将 tokenize 函数应用于数据集的 "text" 列，生成新的特征列。随后，它将标签列 "label" 的数据类型转换为整型并重命名为 "labels"。最后，也是非常重要的一步，函数使用 padded_batch 方法将数据集分批，并对批次内长度不一的序列进行填充（padding），使得每个批次中的所有样本在 input_ids、token_type_ids 和 attention_mask 这些维度上都具有相同的长度，填充值由 tokenizer.pad_token_id 和0指定。

In [3]:
import mindspore
import numpy as np
from mindspore.dataset import transforms

def process_dataset(dataset, tokenizer, max_seq_len=256, batch_size=32, shuffle=False, take_len=None):
    # The tokenize function
    def tokenize(text):
        tokenized = tokenizer(text, truncation=True, max_length=max_seq_len)
        return tokenized['input_ids'], tokenized['token_type_ids'], tokenized['attention_mask']

    # Shuffle the order of the dataset
    if shuffle:
        dataset = dataset.shuffle(buffer_size=batch_size)

    # Select the first several entries of the dataset
    if take_len:
        dataset = dataset.take(take_len)

    # Apply the tokenize function, transforming the 'text' column into the three output columns generated by the tokenizer.
    dataset = dataset.map(operations=[tokenize], input_columns="text", output_columns=['input_ids', 'token_type_ids', 'attention_mask'])
    # Cast the datatype of the 'label' column to int32 and rename the column to 'labels'
    dataset = dataset.map(operations=transforms.TypeCast(mindspore.int32), input_columns="label", output_columns="labels")
    # Batch the dataset with padding.
    dataset = dataset.padded_batch(batch_size, pad_info={'input_ids': (None, tokenizer.pad_token_id),
                                                         'token_type_ids': (None, 0),
                                                         'attention_mask': (None, 0)})
    return dataset

首先从 mindnlp.transformers 库导入并加载了"google/canine-s" 的预训练 CanineTokenizer 分词器实例。然后，定义了批处理大小 batch_size 为 4，并计算了一个 take_len 变量（值为 800），用于从完整训练数据集中选取一个子集。接着，代码调用了先前定义的 process_dataset 函数，分别对 imdb_train 和 imdb_test 数据集进行预处理：对于训练集，它创建了一个名为 small_dataset_train 的小型、打乱顺序并按批次处理的数据集，其中包含了 take_len（即800）条样本；对于测试集，创建了 small_dataset_test，但只选取了400条样本进行打乱和批处理，这两个经过处理的小型数据集随后可以用于模型的快速训练和评估。

In [4]:
from mindnlp.transformers import CanineTokenizer
tokenizer = CanineTokenizer.from_pretrained("google/canine-s")
batch_size = 4
take_len = batch_size * 25
small_dataset_train = process_dataset(imdb_train, tokenizer, batch_size=batch_size, shuffle=True, take_len=take_len)
small_dataset_test = process_dataset(imdb_test, tokenizer, batch_size=batch_size, shuffle=True, take_len=50)

/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/mindnlp/transformers/tokenization_utils_base.py:1526: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted, and will be then set to `False` by default. 
  warnings.warn(


## 四、模型训练

首先从 mindnlp.transformers 库中导入了 CanineConfig 类和 CanineForSequenceClassification 类，这两个类分别用于定义CANINE模型的配置和构建序列分类模型。接着，代码创建了一个 CanineConfig 的实例，并详细设置了模型的各项超参数，如隐藏层大小、注意力头数量、激活函数、dropout概率以及针对当前二分类任务的标签数量（num_labels=2）等。最后，使用这个配置好的 config 对象实例化了一个 CanineForSequenceClassification 模型，这个模型将基于CANINE架构并适用于序列分类任务，并通过 print(type(model)) 打印出所创建模型的类型，以确认模型已成功加载。

In [5]:
from mindnlp.transformers import CanineConfig, CanineForSequenceClassification

config = CanineConfig(
                    hidden_size=32,
                    num_hidden_layers=2,
                    num_attention_heads=4,
                    intermediate_size=37,
                    hidden_act="gelu",
                    hidden_dropout_prob=0.1,
                    attention_probs_dropout_prob=0.1,
                    max_position_embeddings=512,
                    type_vocab_size=16,
                    num_labels=2,  # 分类任务的类别数
                )
model = CanineForSequenceClassification(config)

print(type(model))

[WARNING] DEVICE(10598,fffebaffe120,python):2025-05-22-12:07:48.245.533 [mindspore/ccsrc/plugin/device/ascend/hal/device/ascend_vmm_adapter.h:147] CheckVmmDriverVersion] Open file /etc/ascend_install.info failed.
[WARNING] DEVICE(10598,fffebaffe120,python):2025-05-22-12:07:48.245.625 [mindspore/ccsrc/plugin/device/ascend/hal/device/ascend_vmm_adapter.h:186] CheckVmmDriverVersion] Driver version is less than 24.0.0, vmm is disabled by default, drvier_version: 23.0.6


<class 'mindnlp.transformers.models.canine.modeling_canine.CanineForSequenceClassification'>


使用 mindnlp.engine 中的 TrainingArguments 类来配置模型训练过程中的各种参数。首先指定了训练过程中的输出目录为当前路径下的 "output" 文件夹，用于保存模型、检查点和日志等。接着，设置了每个设备上的训练批次大小和评估批次大小均为1，学习率为2e-5，总训练轮次为6轮。此外，还配置了日志记录的频率（每200步记录一次），并设定评估策略和保存策略均为在每个训练轮次（epoch）结束时执行。这些参数共同定义了模型训练的行为和流程。

In [6]:
from mindnlp.engine import TrainingArguments
training_args = TrainingArguments(
    "./output",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    learning_rate=2e-5,
    num_train_epochs=1,
    logging_steps=50,
    evaluation_strategy='epoch',
    save_strategy='epoch'
)

代码定义了一个用于在模型评估过程中计算指标（具体来说是准确率）的函数 compute_metrics。首先，它从 evaluate 库加载了 "accuracy"（准确率）评估指标，并从 mindnlp.engine.utils 导入了 EvalPrediction 类型，该类型用于封装模型的预测输出和真实标签。然后，compute_metrics 函数接收一个 EvalPrediction 对象（包含模型的原始输出 logits 和真实 labels），通过 np.argmax 函数找到 logits 中概率最高的类别作为预测结果，最后利用加载的 metric 对象计算并返回预测结果与真实标签之间的准确率。

In [7]:
import evaluate
import numpy as np
from mindnlp.engine.utils import EvalPrediction

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred: EvalPrediction):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

代码初始化了一个 mindnlp.engine 中的 Trainer 对象。

In [8]:
from mindnlp.engine import Trainer
trainer = Trainer(
    model=model,
    train_dataset=small_dataset_train,
    eval_dataset=small_dataset_test,
    compute_metrics=compute_metrics,
    args=training_args,
)

In [9]:
trainer.train()

  0%|          | 0/25 [00:00<?, ?it/s]

......

  0%|          | 0/13 [00:00<?, ?it/s]

{'eval_loss': 0.6978093981742859, 'eval_accuracy': 0.0, 'eval_runtime': 8.6984, 'eval_samples_per_second': 1.495, 'eval_steps_per_second': 1.495, 'epoch': 1.0}
{'train_runtime': 89.1989, 'train_samples_per_second': 1.121, 'train_steps_per_second': 0.28, 'train_loss': 0.694559326171875, 'epoch': 1.0}


TrainOutput(global_step=25, training_loss=0.694559326171875, metrics={'train_runtime': 89.1989, 'train_samples_per_second': 1.121, 'train_steps_per_second': 0.28, 'train_loss': 0.694559326171875, 'epoch': 1.0})

## 五、模型评估

使用训练好的CANINE模型对新的单条文本 "I absolutely love this movie" 进行情感预测。它首先利用之前加载的 tokenizer 将文本转换为模型所需的数值化输入，并确保其长度符合要求且转换为MindSpore的 Tensor 格式，同时增加了一个批次维度。随后，代码将模型设置为评估模式，并将处理好的输入传递给模型以获取预测结果。运行结果 SequenceClassifierOutput(loss=None, logits=Tensor(shape=[1, 2], dtype=Float32, value=[[-0.149447203, -0.195080027]]), hidden_states=None, attentions=None) 表明模型成功输出了 logits，这是一个包含两个值的张量，分别代表模型预测该文本属于两个类别（例如负面和正面）的原始分数；在这个具体的例子中，第一个值略高于第二个值，暗示模型可能将该积极情感的文本归类为与第一个索引对应的类别。

In [10]:
import numpy as np
from mindspore import Tensor, ops

text = "I absolutely love this movie"

# Tokenize the text
inputs = tokenizer(text, padding=True, truncation=True, max_length=256)
ts_inputs = {key: Tensor(val).expand_dims(0) for key, val in inputs.items()}

# Predict
model.set_train(False)
outputs = model(**ts_inputs)
print(outputs)

SequenceClassifierOutput(loss=None, logits=Tensor(shape=[1, 2], dtype=Float32, value=
[[-1.98532678e-02, -1.04218852e-02]]), hidden_states=None, attentions=None)


首先通过 ops.softmax 函数将模型输出的 logits 转换为概率分布，然后从中提取每个类别的概率值，并使用 argmax() 确定概率最高的类别索引。接着，利用一个预设的 labels 字典（0代表'neg'，1代表'pos'）将此索引转换为对应的文本标签。最后，代码打印出计算得到的负面情感概率（0.5114）和正面情感概率（0.4886），以及最终预测的类别为 'neg'（对应索引0）。尽管输入文本 "I absolutely love this movie" 明显是正面情感，但运行结果显示模型预测其为负面，这说明模型在此特定样本上的预测与预期不符，可能需要进一步优化或分析。

In [11]:
# Convert predictions to probabilities
predictions = ops.softmax(outputs.logits)
probabilities = predictions.numpy().flatten()

# Get predicted class index
pred_class_idx = probabilities.argmax()

# Map predicted class index to label
labels = {0: 'neg', 1: 'pos'}
pred_class_label = labels[pred_class_idx]

# Output the probabilities and the predicted class label
print(f"Negative sentiment: {probabilities[0]:.4f}")
print(f"Positive sentiment: {probabilities[1]:.4f}")
print(f"Predicted class: {pred_class_label}")
print(f"Predicted label: {pred_class_idx}")

Negative sentiment: 0.4976
Positive sentiment: 0.5024
Predicted class: pos
Predicted label: 1
